In [1]:
import random
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import os 
import math
from tensorflow.keras import Sequential, Model   # 按順序建立的神經網路
from tensorflow.keras import metrics
from tensorflow.keras.layers import Dense,Conv2D,Dropout,Flatten,MaxPooling2D, LSTM, TimeDistributed # Dense全連線層
from tensorflow.keras.layers import RepeatVector, Dense, Flatten, Input, Dropout, Conv1D, Lambda, GRU, Softmax, MaxPooling1D, multiply
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import datetime
from sklearn.metrics import classification_report, confusion_matrix
import itertools
from keras import regularizers
import glob

In [2]:
 
def plot_cm(testY, testX_pred_i_LSTM):
    from sklearn.metrics import classification_report,confusion_matrix
    y_pred=np.argmax(testX_pred_i_LSTM, axis=1)
    y_test=np.argmax(testY, axis=1)
    cm = confusion_matrix(y_test, y_pred)

    print(cm)
    print(classification_report(y_test,y_pred,digits=4))
    y_pred=np.argmax(testX_pred_i_LSTM, axis=1)
    y_test=np.argmax(testY, axis=1)
    cm = confusion_matrix(y_test, y_pred)

    class_names = ['1','2','3']
    plot_confusion_matrix(cm, class_names)
    plot_confusion_matrix(cm, class_names, normalize=False)
    
def plot_confusion_matrix(cm, classes=['1','2','3'],
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):   #plt.cm.Blues、plt.cm.bone_r
        """
        This function prints and plots the confusion matrix.
        Normalization can be applied by setting `normalize=True`.
        """
        from sklearn.metrics import classification_report
        import itertools
        import matplotlib.pyplot as plt
        from sklearn.metrics import confusion_matrix
        import numpy as np
        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            print("Normalized confusion matrix")
        else:
            print('Confusion matrix, without normalization')

        print(cm)
        plt.figure(figsize = (7, 7))
        plt.imshow(cm, interpolation='none', cmap=cmap)
        plt.title(title,size=20)
        plt.colorbar(aspect=5)
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45, size=18)
        plt.yticks(tick_marks, classes, size=18)

        fmt = '.4f' if normalize else 'd'
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(cm[i, j], fmt), fontsize=18,
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

        plt.ylabel('True label',size=20)
        plt.xlabel('Predicted label',size=20)
        plt.tight_layout()


# 必須匯入母體DATA才會有裡面的值
def minmax_(x):
    x_mnmx = pd.DataFrame(columns=['0','1','2','3','4','5','6','7','8','9',
                                       '10','11','12','13','14','15','16','17'])
    for i in range(18):

        DXi = data.X[:,i]
        mx = max(DXi)
        mn = min(DXi)        
        xn = (x.iloc[:,i] -mn) / (mx-mn)
        x_mnmx[f"{i}"] = xn
        
        
    print(x_mnmx.shape)
    return x_mnmx

In [4]:
train_dir = pd.read_csv("../Training/train_dir.csv")
test_dir = pd.read_csv("../Testing/test_dir.csv")
print(len(train_dir), len(test_dir))

678 170


In [7]:
from tensorflow.keras.models import load_model
import joblib
RF_model = joblib.load('/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/MODEL_DATA_10_AllNormal_大於600_by_journey/model_3class_05s/trained_model/RF_model_fromrnn')

# LSTM_model = load_model('/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/MODEL_DATA_10_AllNormal_大於600_by_journey/model_3class_05s/trained_model/LSTM_model.h5') 

# 對每個旅次做預測

In [11]:
import Dataset
from  keras.utils import to_categorical

start = time.time()
# ---main---
data = Dataset.Data_set('X.csv', 'Y_level3_05s.csv')  #呼叫母體  用來做正規

count_=0
test_dir_ = test_dir.values
test_dir_ = test_dir_.reshape(-1,)
#取 測試級路徑
for i in test_dir_:
    a = os.path.split(i)[0]
    print(a)
    x = pd.read_csv(a + '/X.csv')  
    y = pd.read_csv(a + '/Y_level3_05s.csv')
    
    unique, counts = np.unique(y, return_counts=True)
    print(dict(zip(unique, counts)));
    # one hot encode y
    yy = y - 1
    y_oh = to_categorical(yy)
    print(x.shape, y_oh.shape)
    #母體的minmax做正規
    x_mm = minmax_(x) 
    x_mma = x_mm.values
    x_n_w = x_mma.reshape(-1,18)
    #預測
    predx = RF_model.predict(x_n_w)
    #儲存結果
    np.save(a + '/results/y_pred_RF_05s.npy', predx)
    #onehot還原
    y_pred=np.argmax(predx, axis=1)
    y_test=np.argmax(y_oh, axis=1)
    report = classification_report(y_test, y_pred,digits=4, output_dict=True)
    df = pd.DataFrame(report).transpose()
    df.to_csv(a + '/results/df_metrics_RF_05s.csv')

    count_ += 1


end = time.time()
print(start, end)
print("執行時間：%f 秒" % (end - start))    
print(f'總共多少旅次: {count_}')

/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31931992/2020-10-17_121
{1: 54, 2: 804, 3: 253}
(1111, 18) (1111, 3)
(1111, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31932404/2020-10-23_153
{1: 1010, 2: 635, 3: 58}
(1703, 18) (1703, 3)
(1703, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31931745/2020-10-28_30
{1: 5, 2: 191, 3: 826}
(1022, 18) (1022, 3)
(1022, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31931992/2020-10-22_133
{1: 463, 2: 693, 3: 194}
(1350, 18) (1350, 3)
(1350, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31932651/2020-09-17_72
{1: 293, 2: 335, 3: 102}
(730, 18) (730, 3)
(730, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31932651/2020-10-12_93
{1: 607, 2: 451, 3: 72}
(1130, 18) (1130

/home/gumna99/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gumna99/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gumna99/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(608, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31931992/2020-10-18_138
{1: 86, 2: 702, 3: 573}
(1361, 18) (1361, 3)
(1361, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31929871/2020-10-18_49
{1: 905, 2: 356, 3: 31}
(1292, 18) (1292, 3)
(1292, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31929871/2020-09-14_49
{1: 208, 2: 989, 3: 266}
(1463, 18) (1463, 3)
(1463, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31929871/2020-09-06_45
{1: 559, 2: 753, 3: 294}
(1606, 18) (1606, 3)
(1606, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31932651/2020-10-15_115
{1: 766, 2: 763, 3: 79}
(1608, 18) (1608, 3)
(1608, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31902662/2020-09-01_142
{1: 1020, 2: 108, 3: 4}
(11

/home/gumna99/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gumna99/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gumna99/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(1198, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31928725/2020-09-08_177
{1: 260, 2: 412, 3: 111}
(783, 18) (783, 3)
(783, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31932370/2020-09-28_259
{1: 307, 2: 294, 3: 78}
(679, 18) (679, 3)
(679, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31932602/2020-10-15_55
{2: 31, 3: 1158}
(1189, 18) (1189, 3)
(1189, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31929871/2020-10-09_58
{1: 347, 2: 269, 3: 134}
(750, 18) (750, 3)


...

(1226, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31932370/2020-09-24_334
{1: 380, 2: 436, 3: 56}
(872, 18) (872, 3)
(872, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31929871/2020-10-08_52
{1: 257, 2: 611, 3: 260}
(1128, 18) (1128, 3)
(1128, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31902118/2020-10-24_301
{1: 469, 2: 1058, 3: 162}
(1689, 18) (1689, 3)
(1689, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31932370/2020-10-05_329
{1: 27, 2: 720, 3: 669}
(1416, 18) (1416, 3)
(1416, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31931992/2020-09-21_129
{1: 421, 2: 867, 3: 156}
(1444, 18) (1444, 3)
(1444, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31902118/2020-10-13_315
{1: 236, 2: 747, 3: 217}


/home/gumna99/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gumna99/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gumna99/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(1002, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31932602/2020-10-20_30
{2: 22, 3: 1227}
(1249, 18) (1249, 3)
(1249, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31931992/2020-09-28_133
{1: 38, 2: 195, 3: 592}
(825, 18) (825, 3)


/home/gumna99/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gumna99/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gumna99/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(825, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31931992/2020-10-17_120
{1: 182, 2: 336, 3: 178}
(696, 18) (696, 3)
(696, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31932651/2020-09-25_65
{1: 826, 2: 282, 3: 30}
(1138, 18) (1138, 3)
(1138, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31931992/2020-10-21_142
{1: 495, 2: 654, 3: 245}
(1394, 18) (1394, 3)
(1394, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31931745/2020-10-23_27
{1: 39, 2: 577, 3: 2062}
(2678, 18) (2678, 3)
(2678, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31932370/2020-10-06_319
{1: 18, 2: 693, 3: 1027}
(1738, 18) (1738, 3)
(1738, 18)
/3T_HD/Neng/智慧機車_data/data_中山_bydriver/Best_10_drivers/DATA_10_AllNormal_大於600_by_journey/31931745/2020-10-16_24
{1: 33, 2: 409, 3: 703}
(114